<span style="color:#8B949E;">
<b>Note de lecture</b> — Notebook issu de tests itératifs (“speed-tests”).  
Le corpus utilisé ici est un <b>dataset de substitution</b> (non aligné client) uniquement pour valider la mécanique (retrieval + évaluation) et dérouler la roadmap.  
Pour le chemin complet, suivre l’ordre 01 → 10 et lire en priorité les sections Markdown.
</span>


# 📊 Benchmark BM25 (Stage 2) — Notebook de présentation

Ce notebook reprend le script **`2_Benchmark_BM25.py`** et le découpe en étapes lisibles, avec :
- ✅ une exécution **pas à pas**
- 📏 des **métriques retrieval** (Recall@k, MRR, nDCG@k)
- 🧾 une **analyse qualitative** (verbatim) pour comprendre faux positifs / faux négatifs

> **Note compatibilité Python :** ce notebook évite les annotations du type `str | None` (Python 3.10+).  
> On utilise `Optional[str]` pour rester compatible avec Python 3.8/3.9.

In [1]:
# 🔍 Vérification rapide de la version Python du kernel
import sys
print(sys.version)

3.9.13 (tags/v3.9.13:6de2ca5, May 17 2022, 16:36:42) [MSC v.1929 64 bit (AMD64)]


## ⚙️ 0) Dépendances

Ce notebook a besoin de :
- `rank_bm25`

Si tu exécutes dans un environnement où le paquet n’est pas installé, décommente la cellule d’installation ci-dessous.

In [2]:
# 📦 Installation (optionnel) — décommente si besoin
# %pip install rank_bm25

## 🧱 1) Construction du corpus (XML ➜ texte)

On parcourt un dossier de fichiers XML et on construit un corpus minimal :
- `doc_id` : identifiant stable (ici : chemin complet)
- `text` : texte extrait

💡 Dans un vrai projet, cette phase serait souvent **externalisée** (normalisation, métadonnées, filtrage métier…).

In [3]:
import os
import xml.etree.ElementTree as ET
from typing import Optional, List, Dict, Any, Tuple

# ✅ Ajuste ce chemin à ton poste
# Windows (comme ton script) :
DATA_ROOT = r"D:\-- Projet RAG Avocats --\data_main\data"

# Linux / WSL2 (exemple) :
# DATA_ROOT = "/mnt/d/-- Projet RAG Avocats --/data_main/data"

def extract_text_from_xml(xml_path: str) -> Optional[str]:
    """
    Extrait de manière robuste tout le texte d'un fichier XML.

    Hypothèse volontairement simple pour un POC pédagogique :
    - itère sur tous les éléments
    - concatène les champs .text
    """
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        texts: List[str] = []
        for elem in root.iter():
            if elem.text:
                t = elem.text.strip()
                if t:
                    texts.append(t)
        return " ".join(texts)
    except Exception:
        return None

documents: List[Dict[str, Any]] = []

if not os.path.exists(DATA_ROOT):
    raise FileNotFoundError(
        f"DATA_ROOT introuvable: {DATA_ROOT}\n"
        "👉 Mets à jour la variable DATA_ROOT (Windows vs WSL2)."
    )

for root_dir, _, files in os.walk(DATA_ROOT):
    for file in files:
        if file.lower().endswith(".xml"):
            full_path = os.path.join(root_dir, file)
            text = extract_text_from_xml(full_path)

            # Filtre minimal : évite les docs trop pauvres (POC)
            if text and len(text) > 200:
                documents.append({"doc_id": full_path, "text": text})

print(f"📚 Corpus chargé : {len(documents)} documents")

📚 Corpus chargé : 4395 documents


## 🔎 2) Indexation BM25

On tokenise simplement en `lower().split()` puis on construit l’index BM25.

> Objectif : observer les limites du **lexical pur** (synonymie, paraphrase, implicite, etc.).

In [4]:
from rank_bm25 import BM25Okapi

def tokenize(text: str) -> List[str]:
    """Tokenisation volontairement simple."""
    return text.lower().split()

# Corpus tokenisé dans le MÊME ordre que documents
corpus_tokens: List[List[str]] = [tokenize(doc["text"]) for doc in documents]

# Construction de l'index BM25
bm25 = BM25Okapi(corpus_tokens)

print("✅ Index BM25 construit")

✅ Index BM25 construit


## 🧪 3) Jeu de questions (benchmark)

Ici, on associe à chaque question :
- une formulation utilisateur réaliste
- une liste de **mots-clés attendus** (proxy de pertinence)

⚠️ Dans un vrai projet, la vérité terrain serait plutôt :
- des `doc_id` pertinents annotés par des juristes, ou
- des passages (chunks) pertinents, pas seulement des mots-clés.

In [5]:
benchmark_queries = [
    {
        "query_id": "Q1",
        "question": "Dans quels cas un CDI peut-il être rompu sans préavis ?",
        "relevant_keywords": ["faute grave", "rupture sans préavis", "L1234"],
    },
    {
        "query_id": "Q2",
        "question": "Qu'est-ce qu'un licenciement pour motif économique ?",
        "relevant_keywords": ["motif économique", "L1233-3"],
    },
    {
        "query_id": "Q3",
        "question": "Un salarié peut-il contester un licenciement ?",
        "relevant_keywords": ["contestation", "sanctions", "irrégularités du licenciement"],
    },
]

print(f"🧾 Questions benchmark : {len(benchmark_queries)}")

🧾 Questions benchmark : 3


## 🔍 4) Recherche BM25

On récupère les scores BM25 et on trie les documents par score décroissant.

In [6]:
def bm25_search(
    query: str,
    documents: List[Dict[str, Any]],
    bm25: BM25Okapi,
    top_k: int = 10,
) -> List[Tuple[Dict[str, Any], float]]:
    """
    Exécute une recherche BM25 sur le corpus.

    Retour :
    - liste de (doc, score) triée du + pertinent au - pertinent (selon BM25)
    """
    query_tokens = tokenize(query)
    scores = bm25.get_scores(query_tokens)

    ranked = sorted(zip(documents, scores), key=lambda x: x[1], reverse=True)
    return ranked[:top_k]

## 📏 5) Métriques retrieval : Recall@k, MRR, nDCG@k

- **Recall@k** : retrouve-t-on *au moins une* source pertinente dans le top-k ?
- **MRR** : à quel rang arrive le **premier** document pertinent ?
- **nDCG@k** : qualité globale du ranking (pertinent en haut = mieux)

💡 On reste volontairement simple : pertinence = présence d’au moins un mot-clé.

In [7]:
import math

def is_relevant(document_text: str, relevant_keywords: List[str]) -> bool:
    """Pertinence proxy : un mot-clé attendu présent dans le texte."""
    text_lower = document_text.lower()
    return any(keyword.lower() in text_lower for keyword in relevant_keywords)

def recall_at_k(results: List[Tuple[Dict[str, Any], float]], relevant_keywords: List[str], k: int) -> int:
    """Recall@k binaire : 1 si un pertinent dans top-k, sinon 0."""
    top_k_docs = results[:k]
    return int(any(is_relevant(doc["text"], relevant_keywords) for doc, _ in top_k_docs))

def reciprocal_rank(results: List[Tuple[Dict[str, Any], float]], relevant_keywords: List[str]) -> float:
    """MRR : 1/rang du 1er pertinent, 0 si aucun."""
    for rank, (doc, _) in enumerate(results, start=1):
        if is_relevant(doc["text"], relevant_keywords):
            return 1.0 / rank
    return 0.0

def ndcg_at_k(results: List[Tuple[Dict[str, Any], float]], relevant_keywords: List[str], k: int) -> float:
    """nDCG@k : mesure la qualité du ranking (pertinent tôt = mieux)."""
    def dcg(res: List[Tuple[Dict[str, Any], float]]) -> float:
        score = 0.0
        for i, (doc, _) in enumerate(res[:k]):
            if is_relevant(doc["text"], relevant_keywords):
                score += 1.0 / math.log2(i + 2)
        return score

    actual = dcg(results)

    # DCG idéal : tous les pertinents en tête (dans la fenêtre top-k)
    # Ici on approxime l'idéal via le nombre de pertinents présents dans top-k
    nb_rel = sum(is_relevant(doc["text"], relevant_keywords) for doc, _ in results[:k])
    ideal = sum(1.0 / math.log2(i + 2) for i in range(nb_rel))

    return actual / ideal if ideal > 0 else 0.0

## 🧮 6) Évaluation globale du benchmark

On calcule les métriques **moyennes** sur toutes les questions.

In [8]:
def evaluate_benchmark(
    queries: List[Dict[str, Any]],
    documents: List[Dict[str, Any]],
    bm25: BM25Okapi,
    k: int = 10,
) -> Dict[str, float]:
    """Lance le benchmark et renvoie les moyennes des métriques."""
    recall_scores: List[int] = []
    mrr_scores: List[float] = []
    ndcg_scores: List[float] = []

    for q in queries:
        results = bm25_search(q["question"], documents, bm25, top_k=k)

        recall_scores.append(recall_at_k(results, q["relevant_keywords"], k))
        mrr_scores.append(reciprocal_rank(results, q["relevant_keywords"]))
        ndcg_scores.append(ndcg_at_k(results, q["relevant_keywords"], k))

    return {
        "Recall@k": sum(recall_scores) / len(recall_scores),
        "MRR": sum(mrr_scores) / len(mrr_scores),
        "nDCG@k": sum(ndcg_scores) / len(ndcg_scores),
    }

results = evaluate_benchmark(benchmark_queries, documents, bm25, k=10)

print("\n=== 📊 RÉSULTATS DU BENCHMARK BM25 ===")
for metric, value in results.items():
    print(f"{metric} : {value:.3f}")


=== 📊 RÉSULTATS DU BENCHMARK BM25 ===
Recall@k : 0.667
MRR : 0.375
nDCG@k : 0.459


## 🧾 7) Analyse qualitative (VERBATIM)

Les métriques donnent une vision globale, mais en juridique on complète souvent par une lecture qualitative :
- comprendre *pourquoi* un résultat est faux positif,
- et *pourquoi* un document attendu n’apparaît pas (faux négatif),
- diagnostiquer les limites lexicales (synonymie, paraphrase, implicite).

In [9]:
def display_results_per_query(
    queries: List[Dict[str, Any]],
    documents: List[Dict[str, Any]],
    bm25: BM25Okapi,
    k: int = 5,
) -> None:
    """
    Affiche les résultats BM25 pour chaque question (verbatim),
    afin d'analyser faux positifs / faux négatifs et la qualité du ranking.
    """
    for q in queries:
        print("\n" + "=" * 100)
        print(f"❓ Question : {q['question']}")
        print("=" * 100)

        results = bm25_search(q["question"], documents, bm25, top_k=k)

        for rank, (doc, score) in enumerate(results, start=1):
            relevant = is_relevant(doc["text"], q["relevant_keywords"])
            flag = "✅ PERTINENT" if relevant else "❌ NON PERTINENT"
            print(f"\n--- Rang {rank} | Score BM25 : {score:.2f} | {flag}")
            print(doc["text"][:400])

display_results_per_query(benchmark_queries, documents, bm25, k=10)


❓ Question : Dans quels cas un CDI peut-il être rompu sans préavis ?

--- Rang 1 | Score BM25 : 20.37 | ❌ NON PERTINENT
LEGIARTI000021925677 LEGI article/LEGI/ARTI/00/00/21/92/56/LEGIARTI000021925677.xml Article L3142-17 MODIFIE 2010-03-04 2016-08-10 AUTONOME Code du travail Partie législative Troisième partie : Durée du travail, salaire, intéressement, participation et épargne salariale Livre Ier : Durée du travail, repos et congés Titre IV : Congés payés et autres congés Chapitre II : Autres congés Section 2 : Co

--- Rang 2 | Score BM25 : 18.30 | ❌ NON PERTINENT
LEGIARTI000051867286 LEGI article/LEGI/ARTI/00/00/51/86/72/LEGIARTI000051867286.xml Article 18-10 VIGUEUR 2025-08-01 2999-01-01 AUTONOME Arrêté du 9 février 2009 relatif aux modalités d'immatriculation des véhicules Arrêté du 9 février 2009 relatif aux modalités d'immatriculation des véhicules CHAPITRE 12 :  L'HABILITATION DES PROFESSIONNELS DE L'AUTOMOBILE Conformément à l'article 8 de l'arrêté d

--- Rang 3 | Score BM25 :

## 🧠 8) Lecture attendue des résultats (guide)

- **Recall@k faible** → faux négatifs (risque critique en juridique)
- **MRR moyenne** → documents pertinents trouvés mais pas assez haut
- **nDCG@k modérée** → ranking global bruité

👉 Typiquement, cela motive ensuite :
- des **filtres métier** (collections, typologies, périmètre)
- puis une couche **sémantique** (embeddings) ou un **hybride** (BM25 + dense via RRF)